<a href="https://colab.research.google.com/github/BRIJNANDA1979/Segmenting-Kelp-Forests-/blob/main/CNN_for_KELP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
pip install rasterio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 46.1 MB/s eta 0:00:00


In [11]:
import pandas as pd
import os
import rasterio

In [9]:
kelp_files_list = os.listdir('/content/drive/MyDrive/kelp_files_only')

In [10]:
not_kelp_files_list = os.listdir('/content/drive/MyDrive/not_kelp_files')

In [12]:
len(kelp_files_list)

3527

In [13]:
len(not_kelp_files_list)              #### total train_satellite files = 3527 + 2109 = 5636

2109

In [14]:
XX = []                        ### Read Raste data of each Band
YY = []
for i in kelp_files_list:
    with rasterio.open('/content/drive/MyDrive/kelp_files_only/' + i) as src:
      print(src.shape)
      print(src.meta)
      print(src.read(1))
      data_array = src.read(1)    ##### _kelp is 1 Band only
      XX.append(data_array)
      YY.append(1)
for i in not_kelp_files_list:
    with rasterio.open('/content/drive/MyDrive/not_kelp_files/' + i) as src:
      print(src.shape)
      print(src.meta)
      print(src.read(1))
      data_array = src.read(1)    ##### _kelp is 1 Band only
      XX.append(data_array)
      YY.append(0)


/usr/local/lib/python3.10/dist-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


Streaming output truncated to the last 5000 lines.
(350, 350)
{'driver': 'GTiff', 'dtype': 'int32', 'nodata': None, 'width': 350, 'height': 350, 'count': 7, 'crs': None, 'transform': Affine(1.0, 0.0, 0.0,
       0.0, 1.0, 0.0)}
[[10635 10614 10596 ...  8193  8309  9812]
 [10631 10625 10600 ...  7963  7859  8541]
 [10624 10633 10591 ...  7799  7802  8300]
 ...
 [ 8791  8938  9078 ... 12678 12365 11951]
 [ 9127  9246  9284 ... 12767 12610 12248]
 [ 9297  9571  9528 ... 12780 12757 12505]]
(350, 350)
{'driver': 'GTiff', 'dtype': 'int32', 'nodata': None, 'width': 350, 'height': 350, 'count': 7, 'crs': None, 'transform': Affine(1.0, 0.0, 0.0,
       0.0, 1.0, 0.0)}
[[15151 14950 15305 ...  7430  7425  7445]
 [14769 15240 14846 ...  7442  7463  7452]
 [14978 15710 15466 ...  7449  7452  7450]
 ...
 [20143 20110 20101 ... 16939 16970 16969]
 [20131 20095 20134 ... 16886 16866 16970]
 [20336 20209 20168 ... 16915 16836 16553]]
(350, 350)
{'driver': 'GTiff', 'dtype': 'int32', 'nodata': None, 'w

In [19]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [22]:
X = np.array(XX)

In [23]:
X.shape

(5636, 350, 350)

In [24]:
#### Standarize the input data....
scaler = MinMaxScaler()
for i in range(X.shape[0]):
    X[i] = scaler.fit_transform(X[i])


In [26]:
xx = X

In [27]:
yy = np.asarray(YY)

In [18]:
yy.shape

(5636,)

In [28]:
xx = xx.reshape(xx.shape[0],xx.shape[1],xx.shape[2],1)

In [29]:
xx.shape

(5636, 350, 350, 1)

In [30]:
yy = yy.reshape(yy.shape[0],1)

In [31]:
yy.shape

(5636, 1)

In [32]:
X = xx
y= yy

In [33]:
import keras

In [34]:
model = keras.models.Sequential()

initializers = {

}

model.add(
    keras.layers.Conv2D(
        24, 5, input_shape=(350,350,1),
        activation='relu',
    )
)
model.add( keras.layers.MaxPooling2D(2) )
model.add(
    keras.layers.Conv2D(
        48, 5, activation='relu',
    )
)
model.add( keras.layers.MaxPooling2D(2) )
model.add(
    keras.layers.Conv2D(
        96, 5, activation='relu',
    )
)
model.add( keras.layers.Flatten() )
model.add( keras.layers.Dropout(0.2) )

model.add( keras.layers.Dense(
    1, activation='sigmoid',
    )
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 346, 346, 24)      624       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 173, 173, 24)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 169, 169, 48)      28848     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 84, 84, 48)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 80, 80, 96)        115296    
                                                                 
 flatten (Flatten)           (None, 614400)            0

In [35]:
from sklearn.model_selection import train_test_split

In [36]:
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=33)

In [37]:
X_train.shape

(4508, 350, 350, 1)

In [38]:
y_train.shape

(4508, 1)

In [39]:
X_test.shape

(1128, 350, 350, 1)

In [40]:
y_test.shape

(1128, 1)

In [41]:
import tensorflow.keras.optimizers as optimizers

optimizer = optimizers.Adam(learning_rate=0.001)

In [42]:
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
# Fit the model
model.fit(X, y, validation_split=0.33, epochs=1, batch_size=100)

25/38 [==================>...........] - ETA: 1:27 - loss: 0.3034 - accuracy: 0.9224

In [ ]:
#model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=1, batch_size=100)

72/72 [==============================] - 449s 6s/step - loss: 36.3731 - accuracy: 0.9372 - val_loss: 0.4164 - val_accuracy: 0.9563


In [ ]:
prediction = model.predict(X_train)

200/200 [==============================] - 43s 215ms/step
